<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/%EC%86%90%EA%B8%80%EC%94%A8_%EC%88%AB%EC%9E%90_%EC%9D%B8%EC%8B%9D_%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EC%B6%94%EB%A1%A0(%ED%95%99%EC%8A%B5%EC%83%9D%EB%9E%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 이미 학습된 매개변수를 사용하여 학습과정 생략 후 추론 과정만 구현함
- 이러한 추론 과정을 신경망의 순전파(forward propagation)이라고 함
- 신경망에서도 기계 학습과 마찬가지로 2단계로 문제를 해결함
  - 학습 : 학습 데이터를 사용해 가중치 매개변수 학습
  - 추론 : 학습한 매개변수를 사용하여 입력 데이터 분류

### MNIST 데이터셋
- 기계 학습 분야에서 아주 유명한 데이터셋
- 0~9 까지의 숫자 이미지로 구성되며, 훈련 이미지 60,000장, 시험 이미지 10,000장
- MNIST의 이미지 데이넌 28*28 크기의 회색조 이미지(1채널)
- 각 픽셀은 0에서 255까지의 값을 가짐
- 각 이미지별 실제 의미하는 숫자가 레이블로 존재

In [ ]:
# coding: utf-8
try:
    import urllib.request
except ImportError:
    raise ImportError('You should use Python 3.x')
import os.path
import gzip
import pickle
import os
import numpy as np
import sys

url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

__file__ = os.pardir
# print(__file__)
dataset_dir = os.path.dirname(os.path.abspath(__file__))
save_file = dataset_dir + "/mnist.pkl"

train_num = 60000
test_num = 10000
img_dim = (1, 28, 28)
img_size = 784


def _download(file_name):
    file_path = dataset_dir + "/" + file_name
    
    if os.path.exists(file_path):
        return

    print("Downloading " + file_name + " ... ")
    urllib.request.urlretrieve(url_base + file_name, file_path)
    print("Done")
    
def download_mnist():
    for v in key_file.values():
       _download(v)
        
def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name
    
    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            labels = np.frombuffer(f.read(), np.uint8, offset=8)
    print("Done")
    
    return labels

def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name
    
    print("Converting " + file_name + " to NumPy Array ...")    
    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, img_size)
    print("Done")
    
    return data
    
def _convert_numpy():
    dataset = {}
    dataset['train_img'] =  _load_img(key_file['train_img'])
    dataset['train_label'] = _load_label(key_file['train_label'])    
    dataset['test_img'] = _load_img(key_file['test_img'])
    dataset['test_label'] = _load_label(key_file['test_label'])
    
    return dataset

def init_mnist():
    download_mnist()
    dataset = _convert_numpy()
    print("Creating pickle file ...")
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)
    print("Done!")

def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
        
    return T
    

def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    """MNIST 데이터셋 읽기
    
    Parameters
    ----------
    normalize : 이미지의 픽셀 값을 0.0~1.0 사이의 값으로 정규화할지 정한다.
    one_hot_label : 
        one_hot_label이 True면、레이블을 원-핫(one-hot) 배열로 돌려준다.
        one-hot 배열은 예를 들어 [0,0,1,0,0,0,0,0,0,0]처럼 한 원소만 1인 배열이다.
    flatten : 입력 이미지를 1차원 배열로 만들지를 정한다. 
    
    Returns
    -------
    (훈련 이미지, 훈련 레이블), (시험 이미지, 시험 레이블)
    """
    if not os.path.exists(save_file):
        init_mnist()
        
    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)
    
    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0
            
    if one_hot_label:
        dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])    
    
    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], dataset['test_label']) 


# if __name__ == '__main__':
#     init_mnist()

In [ ]:
# MNIST 데이터셋을 내려받아 그 이미지를 넘파이 배열로 변화해주는 파이썬 스크립트 사용
import sys, os
sys.path.append(os.pardir)
# from dataset.mnist import load_mnist
# load_mnist()

(x_train, t_train), (x_test, t_test) = \
  load_mnist(flatten=True, normalize=False)

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


In [ ]:
# MNIST 이미지를 화면에 표출
import numpy as np
from PIL import Image

def img_show(img):
  pil_img = Image.fromarray(np.uint8(img))
  pil_img.show()

img = x_train[0]
label = t_train[0]
print(label)

img = img.reshape(28,28)
print(img.shape)
img_show(img)

5
(28, 28)


In [ ]:
import pickle 

def get_data() :
  (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False, one_hot_label=False)
  return x_test, t_test

def init_network():
  with open('sample_weight.pkl', 'rb') as f:
    network = pickle.load(f)
  return network

def sigmoid(x) :
  return 1 / (1+np.exp(-x)  )

def softmax(x):
  c = np.max(x)
  exp_a = np.exp(x-c)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a

  return y

def predict(network, x) :
  W1, W2, W3 = network['W1'], network['W2'], network['W3']
  b1, b2, b3 = network['b1'], network['b2'], network['b3']

  a1 = np.dot(x, W1) + b1
  z1 = sigmoid(a1)
  a2 = np.dot(z1, W2) + b2
  z2 = sigmoid(a2)
  a3 = np.dot(z2, W3) + b3
  y = softmax(a3)

  return y

In [ ]:
# # sample_weight.pkl 다운로드 후 파일 업로드
# url = "https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch03/sample_weight.pkl"

In [ ]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)) :
  y = predict(network, x[i])
  p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스
  if p == t[i] :
    accuracy_cnt += 1

print('Accuracy : ', str(float(accuracy_cnt / len(x))))

..:13: RuntimeWarning: overflow encountered in exp


Accuracy :  0.9207


### 배치 처리
- MNIST와 같은 이미지 데이터는 하나 당 784개의 원소를 가지고 있음
- 이러한 이미지를 한번에 여러 장 입력하는 경우에는 입력 데이터의 형상이 100*784와 같을 것임 

    ==> 이렇게 하나에 데이터로 묶은 입력 데이터를 '배치(batch)'라고 함

- 배치 처리의 장점 : 이미지 1장당 처리 시간을 줄여줌
  - 1) 수치 계산 라이브러리들은 대부분 크기가 큰 배열 처리에 대한 고도의 최적화가 이루어져 있음
  - 2) 데이터를 읽고 쓰는 느린 I/O속도보다 빠른 CPU나 GPU로 순수 계산을 수행함으로써, 분할된 작은 배열을 여러 번 계산하는 것보다 큰 배열을 한꺼번에 계산하는 것이 빠름

In [29]:
# 배치 처리 구현해보기
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size) :
  x_batch = x[i:i+batch_size]
  y_batch = predict(network, x_batch)
  p = np.argmax(y_batch, axis=1) # 첫번째 차원(100*10의 배열 중 첫번째 차원을 구성하는 각 원소에서 최댓값 인덱스를 찾도록, 차원도 0부터 시작)
  accuracy_cnt += np.sum( p == t[i:i+batch_size])

print('Accuracy : ', str(float(accuracy_cnt / len(x))))

Accuracy :  0.9207


..:13: RuntimeWarning: overflow encountered in exp
